In [1]:
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import math
import random

MIN_LIST_LEN = 6
MAX_LIST_LEN = 6
MAX_STEPS = 120
SUCCESS_REWARD = 10.0
EPS_START = 0.5
EPS_END = 0.05
EPS_DECAY = 1000
GAMMA = 0.98
NUM_EPISODES = 100000
EPISODES_SAVE = 1000
OUTPUT_DIR = 'datasets/rl_sort_transformer_easy/list6_transformer3_128_gamma098_step120'

# Define the vocabulary
vocab = {
    'Comparison': 0,
    'Swap': 1,
    '0': 2,
    '1': 3,
    '2': 4,
    '3': 5,
    '4': 6,
    '5': 7,
    '6': 8,
    '7': 9,
    'less': 10,
    'equal': 11,
    'more': 12,
    'len1': 13,
    'len2': 14,
    'len3': 15,
    'len4': 16,
    'len5': 17,
    'len6': 18,
    'len7': 19,
    'len8': 20,
}
inv_vocab = {v: k for k, v in vocab.items()}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the environment
class SortingEnv:
    def __init__(self):
        self.max_steps = MAX_STEPS

    def reset(self):
        self.length = random.randint(MIN_LIST_LEN, MAX_LIST_LEN)
        self.list = [random.randint(1, 100) for _ in range(self.length)]
        while self.list == sorted(self.list):
            self.list = [random.randint(1, 100) for _ in range(self.length)]
        self.indices = None
        self.current_step = 0
        self.done = False
        initial_token = 'len{}'.format(self.length)
        return vocab[initial_token], self.list.copy()
    
    def get_list(self):
        return self.list
    
    def get_list_len(self):
        return len(self.list)

    def step(self, action_tokens):
        action = action_tokens[0]
        reward = -0.01  # default penalty
        response_token = None

        if action == vocab['Comparison']:
            if len(action_tokens) != 3:
                reward = -1.0
                self.done = True
                return response_token, reward, self.done, self.list.copy()
            index1 = action_tokens[1] - vocab['0']
            index2 = action_tokens[2] - vocab['0']
            if index1 >= self.length or index2 >= self.length or index1 < 0 or index2 < 0:
                reward = -1.0
                self.done = True
                return response_token, reward, self.done, self.list.copy()
            self.indices = (index1, index2)
            if self.list[index1] < self.list[index2]:
                response_token = vocab['less']
                reward = -0.01
            elif self.list[index1] == self.list[index2]:
                response_token = vocab['equal']
                reward = -0.02
            else:
                response_token = vocab['more']
                reward = -0.01
        elif action == vocab['Swap']:
            if self.indices is None:
                reward = -1.0
                self.done = True
                return response_token, reward, self.done, self.list.copy()
            index1, index2 = self.indices
            prev_list = self.list.copy()
            self.list[index1], self.list[index2] = self.list[index2], self.list[index1]
            if self.list == sorted(self.list):
                reward = SUCCESS_REWARD
                self.done = True
            #elif prev_list[index1] > prev_list[index2] and self.list[index1] <= self.list[index2]:
            #    reward = 0.1
            elif (index1 < index2 and prev_list[index1] > prev_list[index2] and self.list[index1] <= self.list[index2]) or \
                (index1 > index2 and prev_list[index1] < prev_list[index2] and self.list[index1] >= self.list[index2]):
                reward = 0.1
            elif (index1 < index2 and prev_list[index1] < prev_list[index2] and self.list[index1] >= self.list[index2]) or \
                (index1 > index2 and prev_list[index1] > prev_list[index2] and self.list[index1] <= self.list[index2]):
                reward = -0.1
            else:
                reward = -0.01
            self.indices = None
        else:
            reward = -1.0
            self.done = True

        self.current_step += 1
        if self.current_step >= self.max_steps:
            self.done = True
        return response_token, reward, self.done, self.list.copy()


Using device: cuda


In [2]:
# Positional Encoding for Transformer
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=256):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=0.1)

        pe = torch.zeros(max_len, d_model)  # (max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)  # (max_len, 1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() *
                             (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)  # Even indices
        pe[:, 1::2] = torch.cos(position * div_term)  # Odd indices
        pe = pe.unsqueeze(1)  # (max_len, 1, d_model)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

# Transformer Model
class TransformerModel(nn.Module):
    def __init__(self, vocab_size, d_model=128, nhead=8, num_layers=3):
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        self.d_model = d_model

        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.decoder = nn.Linear(d_model, vocab_size)

        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.embedding.weight, -initrange, initrange)
        nn.init.zeros_(self.decoder.bias)
        nn.init.uniform_(self.decoder.weight, -initrange, initrange)

    def forward(self, src):
        src = self.embedding(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src)
        output = self.decoder(output)
        return output

def decode(input_tokens, inv_vocab):
    return ' '.join([inv_vocab[x] for x in input_tokens])


def save_checkpoint(model, optimizer, episode, folder, filename):
    """
    Save the model and optimizer state to the designated filepath.

    Args:
        model (nn.Module): The model to save.
        optimizer (torch.optim.Optimizer): The optimizer whose state to save.
        episode (int): The current episode number.
        filepath (str): The path where to save the checkpoint.
    """
    filepath = os.path.join(folder, filename)
    # Ensure the directory exists
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    # Save the checkpoint
    torch.save({
        'episode': episode,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, filepath)
    print(f"Checkpoint saved at episode {episode} to {filepath}")

def load_checkpoint(filepath, model, optimizer):
    """
    Load the model and optimizer state from the designated filepath.

    Args:
        filepath (str): The path from where to load the checkpoint.
        model (nn.Module): The model into which to load the state_dict.
        optimizer (torch.optim.Optimizer): The optimizer into which to load the state.

    Returns:
        int: The episode number to resume from.
    """
    if os.path.isfile(filepath):
        checkpoint = torch.load(filepath, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        episode = checkpoint['episode']
        print(f"Checkpoint loaded from {filepath}, resuming from episode {episode}")
        return episode
    else:
        print(f"No checkpoint found at {filepath}, starting from scratch.")
        return 0

In [ ]:
# Training Loop
def train(verbose=False):
    torch.autograd.set_detect_anomaly(True)  # Detect anomalies in autograd
    vocab_size = len(vocab)
    model = TransformerModel(vocab_size).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)  # Reduced learning rate
    #load_checkpoint("datasets/rl_sort_transformer_easy/list6_transformer3_128_gamma098/ckpt_7000_0.9286.pth", model, optimizer)

    episode_cnt = 0
    total_reward = 0.0
    for episode in range(NUM_EPISODES):
        model.train()  # Set model to training mode
        env = SortingEnv()
        initial_token_id, current_list = env.reset()
        input_tokens = [initial_token_id]
        log_probs = []
        rewards = []
        state = 'expect_action'
        done = False

        while not done and len(input_tokens) < env.max_steps:
            if verbose:
                print(decode(input_tokens, inv_vocab))
                print(env.get_list())
            # Prepare input tensor
            input_seq = torch.tensor(input_tokens, dtype=torch.long, device=device).unsqueeze(1)  # (seq_len, batch_size)
            # Get model output
            output = model(input_seq)  # (seq_len, batch_size, vocab_size)
            # Get logits for the last token
            logits = output[-1, 0, :]  # (vocab_size)

            # Check for NaNs in logits
            if torch.isnan(logits).any():
                print(f"Episode {episode}, NaNs in logits before masking.")
                break

            # Get valid tokens based on state
            def get_valid_tokens(state):
                action_tokens = [vocab['Comparison'], vocab['Swap']]
                index_tokens = [vocab[str(i)] for i in range(env.length)]
                if state == 'expect_action':
                    return action_tokens
                elif state == 'expect_index1':
                    return index_tokens[:-1]
                elif state == 'expect_index2':
                    return [x for x in index_tokens if x > input_tokens[-1]]
                else:
                    # Handle unexpected states by defaulting to expect_action
                    return action_tokens

            valid_token_ids = get_valid_tokens(state)

            # Ensure valid_token_ids are within the vocab range
            if any(idx >= vocab_size or idx < 0 for idx in valid_token_ids):
                print(f"Episode {episode}, invalid indices in valid_token_ids: {valid_token_ids}")
                break

            # Mask invalid tokens
            mask_value = -1e9  # Use a large negative value instead of -inf
            mask = torch.full_like(logits, mask_value).to(device)
            mask[valid_token_ids] = 0
            masked_logits = logits + mask
            
            # Sample action. Have some chance to randomly pick a valid action.
            eps_threshold = EPS_END + (EPS_START - EPS_END) * np.exp(-1.0 * episode / EPS_DECAY)
            if random.random() < eps_threshold:
                masked_logits = masked_logits / 4

            # Check for NaNs in masked_logits
            if torch.isnan(masked_logits).any():
                print(f"Episode {episode}, NaNs in masked_logits after masking.")
                break

            # Compute probabilities
            probs = F.softmax(masked_logits, dim=0)

            # Check for NaNs in probs
            if torch.isnan(probs).any():
                print(f"Episode {episode}, NaNs in probs after softmax.")
                break

            try:
                m = torch.distributions.Categorical(probs)
                action_token = m.sample()
                log_prob = m.log_prob(action_token)
            except ValueError as e:
                print(f"Episode {episode}, error in sampling action: {e}")
                break

            log_probs.append(log_prob)
            input_tokens.append(action_token.item())

            action = action_token.item()
            reward = 0.0
            if state == 'expect_action':
                if action == vocab['Comparison']:
                    state = 'expect_index1'
                elif action == vocab['Swap']:
                    if env.indices is None:
                        # Can't perform Swap without a previous Comparison
                        reward = -1.0
                        rewards.append(reward)
                        done = True
                        continue
                    action_tokens = [vocab['Swap']]
                    response_token, reward, done, current_list = env.step(action_tokens)
                    if verbose:
                        print("Reward:", reward)
                    #rewards.append(reward)
                    state = 'expect_action'
                else:
                    # Invalid action, end the episode
                    reward = -1.0
                    #rewards.append(reward)
                    done = True
            elif state == 'expect_index1':
                index1_token = action_token
                state = 'expect_index2'
            elif state == 'expect_index2':
                index2_token = action_token
                action_tokens = [vocab['Comparison'], index1_token.item(), index2_token.item()]
                response_token, reward, done, current_list = env.step(action_tokens)
                if verbose:
                    print("Reward:", reward)
                #rewards.append(reward)
                if response_token is not None:
                    input_tokens.append(response_token)
                state = 'expect_action'
            else:
                # Invalid state, end the episode
                reward = -1.0
                #rewards.append(reward)
                done = True
            #
            rewards.append(reward)
        #
        # Save checkpoint
        if episode > 0 and episode % EPISODES_SAVE == 0:
            avg_reward = total_reward / (episode_cnt + 0.00001)
            episode_cnt = 0
            total_reward = 0.0
            save_checkpoint(model, optimizer, episode, OUTPUT_DIR, f"ckpt_{episode}_{avg_reward:.4f}.pth")
        #
        #print(decode(input_tokens, inv_vocab))
        #print(env.get_list())
        #print("rewards:", rewards)
        assert len(log_probs) == len(rewards), "log_probs and returns have different sizes!"
        
        if len(log_probs) == 0:
            continue  # Skip if no actions were taken

        # Compute returns
        returns = []
        R = 0
        gamma = GAMMA
        for r in rewards[::-1]:
            R = r + gamma * R
            returns.insert(0, R)
        returns = torch.tensor(returns)
        #print(returns)

        # Check for NaNs in returns
        if torch.isnan(returns).any():
            print(f"Episode {episode}, NaNs in returns.")
            continue

#         if returns.std() != 0 and not torch.isnan(returns.std()):
#             returns = (returns - returns.mean()) / (returns.std() + 1e-9)
#         else:
#             pass #returns = returns - returns.mean()

        # Check for NaNs in standardized returns
        if torch.isnan(returns).any():
            print(f"Episode {episode}, NaNs in standardized returns.")
            continue

        # Compute loss
        loss = 0
        #print("log_probs:", log_probs)
        #print("returns:", returns)
        assert len(log_probs) == len(returns), "log_probs and returns have different sizes!"
        for log_prob, R in zip(log_probs, returns):
            loss -= log_prob * R

        # Check for NaNs in loss
        if torch.isnan(loss):
            print(f"Episode {episode}, NaN in loss.")
            continue

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
        optimizer.step()

        episode_cnt += 1
        total_reward += sum(rewards)
        if episode % 1 == 0:
            print(f"Episode {episode}, loss: {loss.item():.4f}, total reward: {sum(rewards):.4f}")

if __name__ == "__main__":
    train(verbose=False)


/home/mcwave/.local/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Episode 0, loss: -0.3107, total reward: -1.0000
Episode 1, loss: -11.1479, total reward: -0.9300
Episode 2, loss: -4.6589, total reward: -1.1100
Episode 3, loss: -26.1625, total reward: -1.2300
Episode 4, loss: -1.8698, total reward: -1.0000
Episode 5, loss: -2.7978, total reward: -1.1100
Episode 6, loss: -4.2947, total reward: -0.9100
Episode 7, loss: -18.5414, total reward: -1.1500
Episode 8, loss: -3.9510, total reward: -0.1900
Episode 9, loss: -1.2770, total reward: -1.0000
Episode 10, loss: -9.7477, total reward: -0.3900
Episode 11, loss: -34.5162, total reward: -1.2100
Episode 12, loss: -6.1891, total reward: -0.2900
Episode 13, loss: -10.5901, total reward: -1.1400
Episode 14, loss: -10.0464, total reward: -0.4900
Episode 15, loss: -3.6458, total reward: -0.1900
Episode 16, loss: -22.5109, total reward: -1.0600
Episode 17, loss: -0.5602, total reward: 0.0100
Episode 18, loss: -3.3806, total reward: -0.1900
Episode 19, loss: -33.6686, total reward: -1.2800
Episode 20, loss: -3.48

Episode 167, loss: 1.3416, total reward: 0.0200
Episode 168, loss: -19.7596, total reward: -1.1300
Episode 169, loss: -24.5095, total reward: -0.8600
Episode 170, loss: 234.2079, total reward: 10.2300
Episode 171, loss: -5.9485, total reward: -0.2900
Episode 172, loss: -0.3839, total reward: 0.1200
Episode 173, loss: 3.2650, total reward: 0.1200
Episode 174, loss: -0.2433, total reward: 0.1300
Episode 175, loss: 1.2813, total reward: 0.1200
Episode 176, loss: 0.2047, total reward: 0.1100
Episode 177, loss: -0.2644, total reward: 0.0200
Episode 178, loss: -3.2346, total reward: 0.0200
Episode 179, loss: 8.3340, total reward: 0.5300
Episode 180, loss: -2.2372, total reward: -1.0000
Episode 181, loss: 6.4319, total reward: 0.3200
Episode 182, loss: -8.1026, total reward: -0.1800
Episode 183, loss: 5.9354, total reward: 0.4300
Episode 184, loss: -2.9575, total reward: -0.0900
Episode 185, loss: 198.7773, total reward: 10.0700
Episode 186, loss: -15.0174, total reward: -1.1400
Episode 187, 

Episode 333, loss: 1.8055, total reward: 0.4300
Episode 334, loss: 267.7041, total reward: 10.4200
Episode 335, loss: 2.1308, total reward: 0.2200
Episode 336, loss: -1.5896, total reward: 0.0200
Episode 337, loss: 3.2949, total reward: 0.2200
Episode 338, loss: 197.0166, total reward: 10.2800
Episode 339, loss: 0.2191, total reward: 0.2200
Episode 340, loss: 311.2283, total reward: 10.4600
Episode 341, loss: 6.3382, total reward: 0.5200
Episode 342, loss: 2.3915, total reward: 0.2000
Episode 343, loss: 265.6197, total reward: 10.3200
Episode 344, loss: 303.3691, total reward: 10.4100
Episode 345, loss: 5.1081, total reward: 0.1200
Episode 346, loss: -10.1673, total reward: -0.7400
Episode 347, loss: 287.4665, total reward: 10.4400
Episode 348, loss: 2.8181, total reward: 0.2200
Episode 349, loss: 276.8141, total reward: 10.0800
Episode 350, loss: 78.0819, total reward: 10.1500
Episode 351, loss: -9.0461, total reward: -0.8400
Episode 352, loss: 6.4302, total reward: 0.4200
Episode 353

Episode 500, loss: 0.9510, total reward: 0.3900
Episode 501, loss: 199.6430, total reward: 10.1800
Episode 502, loss: 6.4992, total reward: 0.3200
Episode 503, loss: 1.7478, total reward: 0.2200
Episode 504, loss: 125.6557, total reward: 10.0300
Episode 505, loss: 7.1281, total reward: 0.7300
Episode 506, loss: 231.2024, total reward: 10.2000
Episode 507, loss: 4.1961, total reward: 0.5300
Episode 508, loss: 153.9921, total reward: 9.9200
Episode 509, loss: 277.7294, total reward: 10.5400
Episode 510, loss: -16.8180, total reward: -0.6300
Episode 511, loss: 4.7742, total reward: 0.4200
Episode 512, loss: -9.5172, total reward: -0.7300
Episode 513, loss: 235.4372, total reward: 10.2100
Episode 514, loss: 191.9195, total reward: 10.4300
Episode 515, loss: -3.6121, total reward: -0.0900
Episode 516, loss: 101.7388, total reward: 10.3400
Episode 517, loss: 5.6329, total reward: 0.3200
Episode 518, loss: 8.4335, total reward: 0.6300
Episode 519, loss: -1.6697, total reward: 0.1100
Episode 5

...

Episode 13320, loss: 4.4179, total reward: 10.5800
Episode 13321, loss: 0.2781, total reward: 10.3400
Episode 13322, loss: 3.0777, total reward: 10.0300
Episode 13323, loss: 8.2284, total reward: 10.2900
Episode 13324, loss: 4.2914, total reward: 10.3900
Episode 13325, loss: 4.8768, total reward: 10.4800
Episode 13326, loss: 10.7341, total reward: 10.6700
Episode 13327, loss: 1.5656, total reward: 10.4700
Episode 13328, loss: 0.6661, total reward: 9.8900
Episode 13329, loss: -0.0247, total reward: 0.0100
Episode 13330, loss: 15.3455, total reward: 10.2800
Episode 13331, loss: 4.6246, total reward: 10.1700
Episode 13332, loss: 11.0253, total reward: 10.3200
Episode 13333, loss: 0.7422, total reward: 0.8300
Episode 13334, loss: 1.5245, total reward: 10.3400
Episode 13335, loss: 6.7287, total reward: 10.1000
Episode 13336, loss: 11.4797, total reward: 10.4500
Episode 13337, loss: 11.3996, total reward: 10.2400
Episode 13338, loss: 1.6154, total reward: 10.4500
Episode 13339, loss: 2.4378,

Episode 13481, loss: 8.3127, total reward: 10.2900
Episode 13482, loss: 5.2229, total reward: 10.4800
Episode 13483, loss: 2.9147, total reward: 9.9700
Episode 13484, loss: 2.3766, total reward: 10.5300
Episode 13485, loss: 0.0589, total reward: 0.4200
Episode 13486, loss: 12.3734, total reward: 10.2800
Episode 13487, loss: 5.4349, total reward: 10.6200
Episode 13488, loss: 3.3143, total reward: 10.3000
Episode 13489, loss: 0.0701, total reward: 0.8300
Episode 13490, loss: 34.5609, total reward: 10.3900
Episode 13491, loss: 1.9700, total reward: 10.3300
Episode 13492, loss: 1.0010, total reward: 0.5200
Episode 13493, loss: 0.0437, total reward: 0.3200
Episode 13494, loss: 0.1373, total reward: 0.8300
Episode 13495, loss: 4.8861, total reward: 10.5500
Episode 13496, loss: 0.7024, total reward: 0.6300
Episode 13497, loss: 1.7677, total reward: 0.8300
Episode 13498, loss: 0.0272, total reward: 10.9600
Episode 13499, loss: 4.3610, total reward: 10.5800
Episode 13500, loss: 21.4169, total r

Episode 13643, loss: 11.1192, total reward: 10.0900
Episode 13644, loss: 0.0013, total reward: 10.0600
Episode 13645, loss: 2.7901, total reward: 10.3600
Episode 13646, loss: 1.6546, total reward: 0.7300
Episode 13647, loss: 34.0094, total reward: 10.2700
Episode 13648, loss: 0.0173, total reward: 0.2200
Episode 13649, loss: -0.0400, total reward: 0.2200
Episode 13650, loss: 0.0583, total reward: 0.3200
Episode 13651, loss: 20.7918, total reward: 10.2000
Episode 13652, loss: 3.8718, total reward: 10.2800
Episode 13653, loss: 0.7948, total reward: 10.5200
Episode 13654, loss: 24.9848, total reward: 10.4100
Episode 13655, loss: 0.1189, total reward: 10.0700
Episode 13656, loss: 0.2710, total reward: 0.3900
Episode 13657, loss: 31.9193, total reward: 10.4500
Episode 13658, loss: 2.3681, total reward: 10.3800
Episode 13659, loss: 1.3041, total reward: 10.2800
Episode 13660, loss: 0.4013, total reward: 10.0400
Episode 13661, loss: 0.0094, total reward: 0.2200
Episode 13662, loss: 2.9244, to

Episode 13805, loss: 0.0224, total reward: 10.6700
Episode 13806, loss: 0.4245, total reward: 10.4700
Episode 13807, loss: 0.4029, total reward: 10.3300
Episode 13808, loss: -0.0011, total reward: 0.4200
Episode 13809, loss: 0.1105, total reward: 0.7300
Episode 13810, loss: 1.8255, total reward: 10.2800
Episode 13811, loss: 0.0296, total reward: 0.3200
Episode 13812, loss: 28.4974, total reward: 10.0600
Episode 13813, loss: 8.4743, total reward: 10.4500
Episode 13814, loss: 0.3337, total reward: 10.2600
Episode 13815, loss: -0.0409, total reward: 0.2500
Episode 13816, loss: 24.3960, total reward: 10.4500
Episode 13817, loss: 0.2280, total reward: 0.2200
Episode 13818, loss: 6.8325, total reward: 10.5500
Episode 13819, loss: 4.2427, total reward: 10.3700
Episode 13820, loss: 0.0607, total reward: 10.1600
Episode 13821, loss: 27.5306, total reward: 10.5000
Episode 13822, loss: 0.6488, total reward: 10.3500
Episode 13823, loss: 0.0072, total reward: 0.8100
Episode 13824, loss: 6.1805, tot

Episode 13966, loss: 43.1509, total reward: 10.1900
Episode 13967, loss: 12.8724, total reward: 10.1500
Episode 13968, loss: 0.0000, total reward: 9.9900
Episode 13969, loss: 19.5241, total reward: 10.5300
